# API Keys

In [1]:
from config import METISAI_API_KEY

# Text Collector

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

loader = TextLoader('../test.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
docs = text_splitter.split_documents(documents)

In [3]:
documents

[Document(metadata={'source': '../test.txt'}, page_content='حکایت شکار رفتن بهلول و هارون\nروزی خلیفه هارون الرشید و جمعی از درباریان به شکار رفته بودند. بهلول با آنها بود در شکارگاه آهویی نمودار شد.\nخلیفه تیری به سوی آهو انداخت ولی به هدف نخورد. بهلول گفت احسنت !!!\nخلیفه غضبناک شد و گفت مرا مسخره می کنی ؟\nبهلول جواب داد :احسنت من برای آهو بود که خوب فرار نمود.\n\nحکایت زیبای بهلول و سوداگر\nروزی سوداگری بغدادی از بهلول سوال نمود من چه بخرم تا منافع زیاد ببرم؟ بهلول جواب داد آهن و پنبه.\nآن مرد رفت و مقداری آهن و پنبه خرید و انبار نمود اتفاقا" پس از چند ماهی فروخت و سود فراوان برد.\nباز روزی به بهلول بر خورد. این دفعه گفت بهلول دیوانه من چه بخرم تا منافع ببرم؟ بهلول این دفعه گفت پیاز بخر و هندوانه.\nسوداگر این دفعه رفت و سرمایه خود را تمام پیاز خرید و هندوانه انبار نمود و پس از مدت کمی تمام پیاز و هندوانه های او پوسید و از بین رفت و ضرر فراوان نمود.\nفوری به سراغ بهلول رفت و به او گفت در اول که از تو مشورت نموده، گفتی آهن بخر و پنبه، نفعی برده. ولی دفعه دوم این چه پیشنهادی بود کردی؟

In [4]:
docs

[Document(metadata={'source': '../test.txt'}, page_content='حکایت شکار رفتن بهلول و هارون\nروزی خلیفه هارون الرشید و جمعی از درباریان به شکار رفته بودند. بهلول با آنها بود در شکارگاه آهویی نمودار شد.\nخلیفه تیری به سوی آهو انداخت ولی به هدف نخورد. بهلول گفت احسنت !!!\nخلیفه غضبناک شد و گفت مرا مسخره می کنی ؟\nبهلول جواب داد :احسنت من برای آهو بود که خوب فرار نمود.'),
 Document(metadata={'source': '../test.txt'}, page_content='حکایت زیبای بهلول و سوداگر\nروزی سوداگری بغدادی از بهلول سوال نمود من چه بخرم تا منافع زیاد ببرم؟ بهلول جواب داد آهن و پنبه.\nآن مرد رفت و مقداری آهن و پنبه خرید و انبار نمود اتفاقا" پس از چند ماهی فروخت و سود فراوان برد.\nباز روزی به بهلول بر خورد. این دفعه گفت بهلول دیوانه من چه بخرم تا منافع ببرم؟ بهلول این دفعه گفت پیاز بخر و هندوانه.\nسوداگر این دفعه رفت و سرمایه خود را تمام پیاز خرید و هندوانه انبار نمود و پس از مدت کمی تمام پیاز و هندوانه های او پوسید و از بین رفت و ضرر فراوان نمود.\nفوری به سراغ بهلول رفت و به او گفت در اول که از تو مشورت نموده، گفتی آهن ب

# Embedings

# Vector Store

In [5]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from pydantic import SecretStr

# Set UP the link
vector_store = Chroma(
    collection_name="test",
    embedding_function=OpenAIEmbeddings(
        model="text-embedding-3-small",
        api_key=SecretStr(METISAI_API_KEY),
        base_url="https://api.metisai.ir/openai/v1",
    ),
)

vector_store.add_documents(docs, ids=[str(i) for i in range(len(docs))])

/tmp/ipykernel_23772/1150853614.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


['0', '1', '2']

# LLM

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=SecretStr(METISAI_API_KEY),
    base_url="https://api.metisai.ir/openai/v1",
    temperature=0.1,
    verbose=True,
)

# Prompt

In [15]:
from langchain import PromptTemplate

# template = """
# You are a fortune teller. These Human will ask you a questions about their life.
# Use following piece of context to answer the question.
# If you don't know the answer, just say you don't know.
# Keep the answer within 2 sentences and concise.

# Context: {context}
# Question: {question}
# Answer:

# """

template = """
لطفا بر اساس داستان مطرح شده به سوال پاسخ بده
در صورتی که اطلاعات کافی وجود ندارد، بگو اطلاعی ندارم و نمیدانم.
کوتاه پاسخ بده.

داستان: {context}
سوال: {question}
جواب:
"""

prompt = PromptTemplate(
  template=template,
  input_variables=["context", "question"]
)

In [16]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
  {"context": vector_store.as_retriever(),  "question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)

In [17]:
result = rag_chain.invoke('آیا بهلول به هارون الرشید گفت آفرین؟')
print(result)

بله، بهلول گفت "احسنت" ولی منظورش برای آهو بود که خوب فرار کرد.


In [18]:
result = rag_chain.invoke('از نگاه بهلول سود در خرید چیست؟')
print(result)

از نگاه بهلول، سود در خرید به عقل و درک صحیح بستگی دارد. او در ابتدا به شخص عاقل مشورت می‌دهد و در نتیجه او سود می‌برد، اما وقتی به او به عنوان دیوانه خطاب می‌شود، مشاوره‌اش نیز بر اساس آن دیوانگی است و منجر به ضرر می‌شود.


In [19]:
result = rag_chain.invoke('از نگاه بهلول رییس جمهور احمدینژاد چجور آدمیست؟')
print(result)

اطلاعی ندارم و نمیدانم.
